In [ ]:
import pandas as pd #untuk mengolah data
import requests #untuk melakukan permintaan HTTP
import comtradeapicall #library untuk mengakses API UN Comtrade

In [ ]:
subscription_key = '<API KEY>' #ganti API key sesuai milik anda
directory = '<SAVE DIR>' #ganti direktori sesuai keinginan anda, untuk menyimpan file yang diunduh

In [ ]:
from datetime import date #mengambil tanggal hari ini
from datetime import timedelta #menghitung selisih hari
today = date.today()
yesterday = today - timedelta(days=1)
lastweek = today - timedelta(days=7)

In [ ]:
years = [2022, 2023, 2024, 2025] #tahun yang ingin diambil datanya, bisa diganti sesuai kebutuhan
flows = ['M', 'X'] #M untuk impor, X untuk ekspor, bisa ditambah atau dikurangi sesuai kebutuhan
codes = ['880230', '880240', '9306'] #kode HS untuk pesawat tempur dan rudal, bisa diganti sesuai kebutuhan
df_list = []

for year in years:
    for flow in flows:
        for code in codes:
            print(f"Tahun {year} | Flow {flow} | Code {code}")

            df_year = comtradeapicall.previewFinalData(
                typeCode='C', #C untuk perdagangan komoditas
                freqCode='A', #A untuk tahunan
                clCode='HS', #HS untuk sistem klasifikasi HS
                period=year,
                reporterCode='360', #360 untuk Indonesia
                cmdCode=code,
                flowCode=flow,
                partnerCode=None, #tidak ada partner khusus
                partner2Code=None, #tidak ada partner2 khusus
                customsCode=None, #tidak ada kode bea cukai khusus
                motCode=None, #tidak ada mode of transport khusus
                maxRecords=5000, #maksimal 5000 baris
                format_output='JSON', #format output JSON
                aggregateBy=None, #tidak ada agregasi khusus
                breakdownMode='classic', #mode breakdown klasik
                countOnly=None, #tidak hanya menghitung
                includeDesc=True #sertakan deskripsi
            )

            if isinstance(df_year, pd.DataFrame) and not df_year.empty:
                df_year['Year'] = year
                df_year['FlowCode'] = flow
                df_year['HSCode'] = code
                df_list.append(df_year)

# Gabung semua data
if df_list:
    df_all = pd.concat(df_list, ignore_index=True)
    print("\nDONE — Data berhasil diambil!")
    print(f"Total rows: {len(df_all)}")
    display(df_all.head())
else:
    print("\nTidak ada data ditemukan sama sekali.")

In [ ]:
import os #untuk operasi sistem seperti membuat direktori

os.makedirs(directory, exist_ok=True) #membuat direktori jika belum ada

df_all.head(None)
filename = f"comtrade_{today}.xlsx" 
filepath = os.path.join(directory, filename)

df_all.to_excel(filepath, index=False) #menyimpan Data ke file XLSX
print(f"File XLSX tersimpan:\n{filepath}")
